In [1]:
from os.path import join, dirname, basename, splitext, abspath
from glob import glob
import pandas as pd
import yaml
import platform
from datetime import timedelta
from IPython.display import display
from IPython.core.display import display, HTML
import platform
from util.loader import load_raw_data, convert_data
display(HTML("<style>.container { width:100% !important; }</style>"))

this_folder = dirname(__file__) if '__file__' in globals() else abspath('')
root_folder = dirname(dirname(this_folder))
def get_test_result_folder(testname = 'baseline'):
    return join(root_folder, 'test', 'testresults', 'XPS-15-9560', testname, 'CUTEst')

def color_negative_red(val):
    color = 'red' if val < 0 else 'black'
    return f'color: {color}'
def color_negative_red_positive_green(val):
    if val > 0:
        return 'color: green'
    elif val < 0:
        return 'color: red'
    else:
        return ''

In [2]:
tests = [
    'panoc-lbfgs-boxnqp',
    'panoc-slbfgs-boxnqp',
    'lbfgspp-boxnqp',
    'lbfgsbpp-boxnqp',
]
results_folders = [ get_test_result_folder(t) for t in tests ]

In [3]:
results_raw = [ load_raw_data(f) for f in results_folders ]

In [4]:
results_df = [ convert_data(r) for r in results_raw ]

In [5]:
def df_stats(df):
    conv = df['status'].value_counts()['Converged']
    tot = df['status'].count()
    tot_time = df['time'].sum()
    conv_time = df[df['status'] == 'Converged']['time'].sum()
    print(f'Converged:      {conv}/{tot} = {100*conv/tot:.02f}%')
    print(f'Total time:     {tot_time:.03f}s')
    print(f'Converged time: {conv_time:.03f}s')

In [6]:
for test, df in zip(tests, results_df):
    print(test + '\n---\n')
    df_stats(df)
    print('\n')

panoc-lbfgs-boxnqp
---

Converged:      138/219 = 63.01%
Total time:     1860.495s
Converged time: 164.391s


panoc-slbfgs-boxnqp
---

Converged:      143/219 = 65.30%
Total time:     1880.364s
Converged time: 227.101s


lbfgspp-boxnqp
---

Converged:      117/219 = 53.42%
Total time:     815.933s
Converged time: 251.718s


lbfgsbpp-boxnqp
---

Converged:      150/219 = 68.49%
Total time:     1070.354s
Converged time: 96.082s




In [7]:
all_converged = None 
for df in results_df:
      all_converged = (df['status'] == 'Converged') if all_converged is None else all_converged & (df['status'] == 'Converged')
print(f"Converged in all tests: {len(results_df[0][all_converged].index)}")

stats = [
    ('Time:                  ', lambda d: d['time'].sum()),
    ('Objective evaluations: ', lambda d: d['f evaluations'].sum()),
    ('Gradient evaluations:  ', lambda d: d['grad_f evaluations'].sum()),
    ('Outer iterations:      ', lambda d: d['outer iterations'].sum()),
    ('Inner iterations:      ', lambda d: d['inner iterations'].sum()),
    ('Linesearch failures:   ', lambda d: d['linesearch failures'].sum()),
    ('L-BFGS failures:       ', lambda d: d['L-BFGS failures'].sum()),
    ('L-BFGS rejected:       ', lambda d: d['L-BFGS rejected'].sum()),
]

print('\t\t\t' + '\t'.join(tests))
for stat, fun in stats:
      res = [ str(fun(df[all_converged])) for df in results_df]
      print(stat + '\t' + '\t'.join(res))

Converged in all tests: 79
			panoc-lbfgs-boxnqp	panoc-slbfgs-boxnqp	lbfgspp-boxnqp	lbfgsbpp-boxnqp
Time:                  	27.461838000000004	35.551465	8.878687999999999	5.214641
Objective evaluations: 	409942	373863	34812	19193
Gradient evaluations:  	363342	316104	34132	18545
Outer iterations:      	770	712	680	648
Inner iterations:      	160001	130252	25561	15379
Linesearch failures:   	4287	5979	0	0
L-BFGS failures:       	0	0	0	0
L-BFGS rejected:       	3109	2094	0	0


In [8]:
df = results_df[0]
df[(df['status'] != 'Converged') & (df['n'] == 2)]

,status,time,inner iterations,outer iterations,inner convergence failures,f,ε,δ,f evaluations,grad_f evaluations,...,grad_g evaluations,linesearch failures,L-BFGS failures,L-BFGS rejected,‖Σ‖,‖x‖,‖y‖,n,m,box constr x
name,,,,,,,,,,,,,,,,,,,,,
HS2NE,MaxIter,0.007519,4243,200,191,0.0,5.5,2.215671,9648,9434,...,9434,37,0,4161,1.081367e+15,1.930766,2.217545e+15,2,2,1
